### The example script below demonstrates how a randomized plate layout can be generated from an initial description (see input_file.csv) of the experiment 

In [1]:
import pandas as pd
from datarail.experimental_design import process_assay as pa
from datarail.experimental_design import hpdd_utils as hu

The input file (see **`input_file.csv`**) should be broad description of the planned experimental design and should contain the following columns<br>
>__agent__ : lists the names of drugs, including positve controls.<br>
**max_dose__um** : lists the highest dose for each agent.<br>
__num_doses__ : lists the number of doses for each agent.<br>
__role__ : lists the intended role for each agent
   `'treatment', or 'positive_control'`<br>
__num_replicates__ : lists number of times a drug's
   dosing scheme is replicated on the same plate.<br>
__exlcude_doses__ :  list doses (comma seperated) to be excluded for a given drug (**OPTIONAL**; defaults to removing lowest doses if num_doses < 9).<br>



In [15]:
# construct mapping between wells and treatment conditions
df_well = pa.construct_well_level_df('input_file.csv')
df_well = pa.add_negative_control(df_well, control_name='DMSO')

In [16]:
df_well[df_well.agent == 'agent_18']['concentration'].unique()

array([10.    ,  3.1623,  1.    ,  0.3162,  0.1   ,  0.0316,  0.01  ])

In [17]:
# define drug and dose to be used as harsh treatment for plate barcode (OPTIONAL)
fingerprint_drug = 'Staurosporine'
fingerprint_dose = 1 # in micromolar

In [19]:
# create randomized plate layout for 3 replicates
plate_names = ['RL_A', 'RL_B', 'RL_C']

In [20]:
dfr = pa.randomize_wells(df_well, plate_names=plate_names,
                         fingerprint_drug=fingerprint_drug, # optional
                         fingerprint_dose=fingerprint_dose) # optional

In [23]:
dfr.head()
dfr.shape

(1152, 6)

In [25]:
# load a file (see drug_stock.csv) with name of the agents and the corresponding stock concentrations
# The name of the agents in the file should match those in design dataframe
df_stock = pd.read_csv('drug_stock.csv')
df_stock.head()

,name,stock_concentration
0,agent_1,10
1,agent_2,10
2,agent_3,10
3,agent_4,10
4,agent_5,10


In [26]:
# If the D300 is going to be used, run the code below to generate the hpdd file
hu.export_hpdd(dfr, df_stock, 'RL_D300.hpdd')